In [2]:
import pandas as pd
from fastparquet import ParquetFile
import duckdb

In [9]:
p1 = ParquetFile("1-8800.parquet").to_pandas()

In [10]:
p2 = ParquetFile("8861-10700.parquet").to_pandas()

In [11]:
pd.concat([p1,p2]).drop_duplicates().to_parquet("first_half.parquet")